In [31]:
import os
import re
import pandas as pd
from tqdm import tqdm

class Zerofucks:
    
    def __init__(self):
        self.df = pd.DataFrame(
            {'file': [], 'line_no': [], 'badword': [], 'content': []}, 
            columns=['file', 'line_no', 'badword', 'content']
        )
    
    def __makeRegex__(self, shitToFind=["fuck", "shit"]):
        '''
        shitToFind is a list of word literals,
        uses word boundary \b

        returns a regex string
        '''
        shitString = ''
        for word in shitToFind:
            # shitString += r"\b" + word + r"\b|"
            
            # case insensitive (?i) has deprecation warning:
            # https://github.com/bottlepy/bottle/issues/949
            shitString += r"(?i)\b" + word + r"\b|"
        return shitString[:-1]
        
    def write_df(self, path='./', filename='out.csv'):
        '''
        path: path to where you want to put the file
        filename: name of file
        
        Writes the csv from self.df to a csv file
        '''
        try:
            self.df.to_csv(path_or_buf=path + filename, index=False, chunksize=1000)
            print("File written to {}".format(path + filename))
        except:
            print("Unable to write csv, did you run .find_shit() yet?")
            
    def erase_shit(self, input_file, output_file, bad_words=['fuck', 'shit']):
        '''
        Erases any bad words from the user specified files.
        '''
        
        print('Replacing bad words in {}...'.format(input_file))
#         try:
        infile = open(input_file, mode='r').readlines()
        outfile = open(output_file, mode='w')
        shit_dict = {}
        for content in tqdm(infile):
            if len(re.findall(self.__makeRegex__(bad_words), content)) > 0:
#                 print('Replacing {}...'.format(re.findall(self.__makeRegex__(bad_words), content)[0]))
                if re.findall(self.__makeRegex__(bad_words), content)[0] in shit_dict:
                    shit_dict[re.findall(self.__makeRegex__(bad_words), content)[0]] += 1
                else:
                    shit_dict[re.findall(self.__makeRegex__(bad_words), content)[0]] = 1
            outfile.write(re.sub(self.__makeRegex__(bad_words), ' ', content))
#         except:
#             print('Unable to replace bad words in that file.')
        return shit_dict
    
    def find_shit(self, root='./', bad_words=['fuck', 'shit'], include_content=True):
        '''
        root: the directory root where you want the crawl to start
        bad_words: list of words you want to search for.
            These can also be regular expressions. It will
            match partials, so 'fuck' will match 'fucking',
            'unfuckingbelievable', etc. Use \b and similar
            to constrain to word boundaries.
        
        The meat and potatoes, this is what conducts the walk
        and writes the resultant dataframe to self.df.
        
        To export the df to a csv, use the write_df() method.
        '''        
        for item in os.walk(root):
            
            # keep records for each directory parsed
            file_df = []
            line_no_df = []
            badword_df = []
            if include_content:
                content_df = []
            
            for file in item[2]:
                print('Searching {}'.format(item[0]+'/'+file))
                try:
                    openfile = open(item[0] + '/' + file, "r").readlines()
                    for line_no, content in enumerate(openfile):
                        if len(re.findall(self.__makeRegex__(bad_words), content)) > 0:
                            for i, badword in enumerate(re.findall(self.__makeRegex__(bad_words), content)):
                                file_df.append(item[0]+'/'+file)
                                line_no_df.append(line_no)
                                badword_df.append(re.findall(self.__makeRegex__(bad_words),content)[i])
                                # rstrip to remove newline character
                                if include_content:
                                    content_df.append(content.rstrip())
                except:
                    pass
                            
#             print(content_df)

            # write the records to the dataframe with each dir parsed
            if include_content:
                self.df = self.df.append(pd.DataFrame({'file': file_df, 
                                'line_no': line_no_df, 
                                'badword': badword_df,
                                'content': content_df},
                                columns=['file', 'line_no', 'badword', 'content']),
                                ignore_index=True)
            else:
                self.df = self.df.append(pd.DataFrame({'file': file_df, 
                    'line_no': line_no_df, 
                    'badword': badword_df},
                    columns=['file', 'line_no', 'badword']),
                    ignore_index=True)
        
        # change the silly auto-detected float line_no to an integer
        self.df['line_no'] = self.df.copy()['line_no'].apply(lambda x: int(x))
        # return self.df
        print("Dataframe successfully created, \n \
            use <obj>.df to print the df, or \n \
            .write_df() method to write to file.")

Find words in files

Use this to find bad words in multiple files

In [32]:
# instantiate class
myFinder = Zerofucks()

# load in bad words list from file,
# write to list my_badwords
my_badwords = []
badwords_file = open('./bad_words.txt', "r").readlines()
for line in badwords_file:
    my_badwords.append(line.rstrip())

In [33]:
# find stuff!
myFinder.find_shit('./course_dump', my_badwords, include_content=False)

/home/greg/miniconda3/envs/ga/lib/python3.6/site-packages/pandas/core/frame.py:6211: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


Searching ./course_dump/11.01-docker-on-aws/docker-on-aws.ipynb
Searching ./course_dump/11.01-docker-on-aws/.git/packed-refs
Searching ./course_dump/11.01-docker-on-aws/.git/index
Searching ./course_dump/11.01-docker-on-aws/.git/HEAD
Searching ./course_dump/11.01-docker-on-aws/.git/description
Searching ./course_dump/11.01-docker-on-aws/.git/config
Searching ./course_dump/11.01-docker-on-aws/.git/refs/heads/master
Searching ./course_dump/11.01-docker-on-aws/.git/refs/remotes/origin/HEAD
Searching ./course_dump/11.01-docker-on-aws/.git/hooks/pre-applypatch.sample
Searching ./course_dump/11.01-docker-on-aws/.git/hooks/update.sample
Searching ./course_dump/11.01-docker-on-aws/.git/hooks/applypatch-msg.sample
Searching ./course_dump/11.01-docker-on-aws/.git/hooks/prepare-commit-msg.sample
Searching ./course_dump/11.01-docker-on-aws/.git/hooks/fsmonitor-watchman.sample
Searching ./course_dump/11.01-docker-on-aws/.git/hooks/post-update.sample
Searching ./course_dump/11.01-docker-on-aws/.git/

Searching ./course_dump/7.05-lesson-cnns_ii/.gitignore
Searching ./course_dump/7.05-lesson-cnns_ii/README.md
Searching ./course_dump/7.05-lesson-cnns_ii/starter-code.ipynb
Searching ./course_dump/7.05-lesson-cnns_ii/solution-code/solution-code.ipynb
Searching ./course_dump/7.05-lesson-cnns_ii/.git/packed-refs
Searching ./course_dump/7.05-lesson-cnns_ii/.git/index
Searching ./course_dump/7.05-lesson-cnns_ii/.git/HEAD
Searching ./course_dump/7.05-lesson-cnns_ii/.git/description
Searching ./course_dump/7.05-lesson-cnns_ii/.git/config
Searching ./course_dump/7.05-lesson-cnns_ii/.git/refs/heads/master
Searching ./course_dump/7.05-lesson-cnns_ii/.git/refs/remotes/origin/HEAD
Searching ./course_dump/7.05-lesson-cnns_ii/.git/hooks/pre-applypatch.sample
Searching ./course_dump/7.05-lesson-cnns_ii/.git/hooks/update.sample
Searching ./course_dump/7.05-lesson-cnns_ii/.git/hooks/applypatch-msg.sample
Searching ./course_dump/7.05-lesson-cnns_ii/.git/hooks/prepare-commit-msg.sample
Searching ./course

Searching ./course_dump/9.07-lesson-pymc3-bayes/solution-code/estimation-with-pymc3-solutions.ipynb
Searching ./course_dump/9.07-lesson-pymc3-bayes/practice-bayes-intro_to_pymc3/.gitignore
Searching ./course_dump/9.07-lesson-pymc3-bayes/practice-bayes-intro_to_pymc3/practice-pymc3-auto-data.ipynb
Searching ./course_dump/9.07-lesson-pymc3-bayes/practice-bayes-intro_to_pymc3/.DS_Store
Searching ./course_dump/9.07-lesson-pymc3-bayes/practice-bayes-intro_to_pymc3/solution-code/practice-pymc3-auto-data-solutions.ipynb
Searching ./course_dump/9.07-lesson-pymc3-bayes/practice-bayes-intro_to_pymc3/datasets/Auto.csv
Searching ./course_dump/9.07-lesson-pymc3-bayes/practice-bayes-intro_to_pymc3/datasets/SkillCraft1_Dataset.csv
Searching ./course_dump/9.07-lesson-pymc3-bayes/practice-bayes-intro_to_pymc3/datasets/description.txt
Searching ./course_dump/9.07-lesson-pymc3-bayes/.git/packed-refs
Searching ./course_dump/9.07-lesson-pymc3-bayes/.git/index
Searching ./course_dump/9.07-lesson-pymc3-bayes

Searching ./course_dump/9.01-recommender-systems-ii/item-item-demo.ipynb
Searching ./course_dump/9.01-recommender-systems-ii/19_recommendation_engines-updated.pptx
Searching ./course_dump/9.01-recommender-systems-ii/.git/packed-refs
Searching ./course_dump/9.01-recommender-systems-ii/.git/index
Searching ./course_dump/9.01-recommender-systems-ii/.git/HEAD
Searching ./course_dump/9.01-recommender-systems-ii/.git/description
Searching ./course_dump/9.01-recommender-systems-ii/.git/config
Searching ./course_dump/9.01-recommender-systems-ii/.git/refs/heads/master
Searching ./course_dump/9.01-recommender-systems-ii/.git/refs/remotes/origin/HEAD
Searching ./course_dump/9.01-recommender-systems-ii/.git/hooks/pre-applypatch.sample
Searching ./course_dump/9.01-recommender-systems-ii/.git/hooks/update.sample
Searching ./course_dump/9.01-recommender-systems-ii/.git/hooks/applypatch-msg.sample
Searching ./course_dump/9.01-recommender-systems-ii/.git/hooks/prepare-commit-msg.sample
Searching ./cour

...

Searching ./course_dump/11.07-outcomes-interview_prep/.git/packed-refs
Searching ./course_dump/11.07-outcomes-interview_prep/.git/index
Searching ./course_dump/11.07-outcomes-interview_prep/.git/HEAD
Searching ./course_dump/11.07-outcomes-interview_prep/.git/description
Searching ./course_dump/11.07-outcomes-interview_prep/.git/config
Searching ./course_dump/11.07-outcomes-interview_prep/.git/refs/heads/master
Searching ./course_dump/11.07-outcomes-interview_prep/.git/refs/remotes/origin/HEAD
Searching ./course_dump/11.07-outcomes-interview_prep/.git/hooks/pre-applypatch.sample
Searching ./course_dump/11.07-outcomes-interview_prep/.git/hooks/update.sample
Searching ./course_dump/11.07-outcomes-interview_prep/.git/hooks/applypatch-msg.sample
Searching ./course_dump/11.07-outcomes-interview_prep/.git/hooks/prepare-commit-msg.sample
Searching ./course_dump/11.07-outcomes-interview_prep/.git/hooks/fsmonitor-watchman.sample
Searching ./course_dump/11.07-outcomes-interview_prep/.git/hooks/po

Searching ./course_dump/10.06-lesson-benfords-law/.git/logs/HEAD
Searching ./course_dump/10.06-lesson-benfords-law/.git/logs/refs/heads/master
Searching ./course_dump/10.06-lesson-benfords-law/.git/logs/refs/remotes/origin/HEAD
Searching ./course_dump/10.06-lesson-benfords-law/datasets/Austin_Finance_Online_eCheckbook.csv
Searching ./course_dump/11.04-lesson-explainable_models-ethics/README.md
Searching ./course_dump/11.04-lesson-explainable_models-ethics/lime.pdf
Searching ./course_dump/11.04-lesson-explainable_models-ethics/explainable-models.ipynb
Searching ./course_dump/11.04-lesson-explainable_models-ethics/assets/chowdhury.png
Searching ./course_dump/11.04-lesson-explainable_models-ethics/assets/okcupid.png
Searching ./course_dump/11.04-lesson-explainable_models-ethics/assets/eu.png
Searching ./course_dump/11.04-lesson-explainable_models-ethics/assets/viz_exp.png
Searching ./course_dump/11.04-lesson-explainable_models-ethics/assets/lime.png
Searching ./course_dump/11.04-lesson-ex

Searching ./course_dump/8.05-pca-intro-lab/.git/packed-refs
Searching ./course_dump/8.05-pca-intro-lab/.git/index
Searching ./course_dump/8.05-pca-intro-lab/.git/HEAD
Searching ./course_dump/8.05-pca-intro-lab/.git/description
Searching ./course_dump/8.05-pca-intro-lab/.git/config
Searching ./course_dump/8.05-pca-intro-lab/.git/refs/heads/master
Searching ./course_dump/8.05-pca-intro-lab/.git/refs/remotes/origin/HEAD
Searching ./course_dump/8.05-pca-intro-lab/.git/hooks/pre-applypatch.sample
Searching ./course_dump/8.05-pca-intro-lab/.git/hooks/update.sample
Searching ./course_dump/8.05-pca-intro-lab/.git/hooks/applypatch-msg.sample
Searching ./course_dump/8.05-pca-intro-lab/.git/hooks/prepare-commit-msg.sample
Searching ./course_dump/8.05-pca-intro-lab/.git/hooks/fsmonitor-watchman.sample
Searching ./course_dump/8.05-pca-intro-lab/.git/hooks/post-update.sample
Searching ./course_dump/8.05-pca-intro-lab/.git/hooks/pre-rebase.sample
Searching ./course_dump/8.05-pca-intro-lab/.git/hooks/

Searching ./course_dump/11.01-lab-spark-model/.git/objects/pack/pack-ffeb66c0c11241a2cb043d1737588424d96d80c5.idx
Searching ./course_dump/11.01-lab-spark-model/.git/info/exclude
Searching ./course_dump/11.01-lab-spark-model/.git/logs/HEAD
Searching ./course_dump/11.01-lab-spark-model/.git/logs/refs/heads/master
Searching ./course_dump/11.01-lab-spark-model/.git/logs/refs/remotes/origin/HEAD
Searching ./course_dump/11.05-lesson-spark-ml/README.md
Searching ./course_dump/11.05-lesson-spark-ml/Machine Learning.html
Searching ./course_dump/11.05-lesson-spark-ml/solution-code/Machine Learning Solution.html
Searching ./course_dump/11.05-lesson-spark-ml/assets/attach.png
Searching ./course_dump/11.05-lesson-spark-ml/assets/upload.png
Searching ./course_dump/11.05-lesson-spark-ml/assets/import.png
Searching ./course_dump/11.05-lesson-spark-ml/.git/packed-refs
Searching ./course_dump/11.05-lesson-spark-ml/.git/index
Searching ./course_dump/11.05-lesson-spark-ml/.git/HEAD
Searching ./course_dump

Searching ./course_dump/6.04-lesson-support_vector_machines-svm/.git/packed-refs
Searching ./course_dump/6.04-lesson-support_vector_machines-svm/.git/index
Searching ./course_dump/6.04-lesson-support_vector_machines-svm/.git/HEAD
Searching ./course_dump/6.04-lesson-support_vector_machines-svm/.git/description
Searching ./course_dump/6.04-lesson-support_vector_machines-svm/.git/config
Searching ./course_dump/6.04-lesson-support_vector_machines-svm/.git/refs/heads/master
Searching ./course_dump/6.04-lesson-support_vector_machines-svm/.git/refs/remotes/origin/HEAD
Searching ./course_dump/6.04-lesson-support_vector_machines-svm/.git/hooks/pre-applypatch.sample
Searching ./course_dump/6.04-lesson-support_vector_machines-svm/.git/hooks/update.sample
Searching ./course_dump/6.04-lesson-support_vector_machines-svm/.git/hooks/applypatch-msg.sample
Searching ./course_dump/6.04-lesson-support_vector_machines-svm/.git/hooks/prepare-commit-msg.sample
Searching ./course_dump/6.04-lesson-support_vect

In [40]:
# write the no-output version to a file
myFinder.df[['file', 'badword', 'line_no']].head()

In [82]:
res = myFinder.df[['file', 'badword', 'line_no']]

In [83]:
res['badword'] = res['badword'].apply(lambda x: x.lower())

/home/greg/miniconda3/envs/ga/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [84]:
res.to_csv('./output_no_content.csv', index=False, chunksize=1000)

Just make a snippet of each offense

In [26]:
myFinder.df.groupby('file', as_index=False).head(5).to_csv('./head5.csv', index=False, chunksize=1000)

Erase badwords from a file and return a 'cleaned' file

Use this to erase badwords from a single file and return a cleaned file

In [66]:
# erase stuff! (returns a dictionary)
out_dict = myFinder.erase_shit('./labeledTrainData.tsv', './out.tsv', my_badwords)
# let's get a report of the stuff we replaced in the erase_shit() call:
out_df = pd.DataFrame.from_dict(out_dict, orient='index')
out_df.columns = ['count']
out_df.sort_values('count', ascending=False)[:10]

  0%|          | 6/24138 [00:00<07:15, 55.38it/s]

Replacing bad words in ./labeledTrainData.tsv...


100%|██████████| 24138/24138 [04:50<00:00, 83.04it/s]


,count
sex,783
crap,606
hell,521
God,440
dog,322
gun,314
god,223
drug,215
damn,186
bloody,172
